In [ ]:
import pandas as pd
import keras
from keras.layers import LSTM, SimpleRNN, Input, Bidirectional, TimeDistributed, Dropout, Dense, Activation, BatchNormalization
from keras.models import Model
from keras.utils.np_utils import to_categorical
from keras.regularizers import l2
import tensorflow as tf
import numpy as np
import os
import fnmatch
from matplotlib import pyplot as plt
from IPython.display import clear_output
from modules.callbacks import PlotLosses
from modules.lstm_encoder import LSTMEncoder

In [ ]:
layers = [
    Dropout(0.2),
    Dense(4096, activation='relu'),
    Dropout(0.2),
    Dense(2048, activation='relu'),
    Dense(2048, activation='relu'),
    Bidirectional(LSTM(1024, return_sequences=True)),
]

## Training

In [ ]:
encoder = LSTMEncoder(
    hidden_layers=layers,
    source_path='./features/lstm/2_steps/',
    weights_path='./convnet_weights/weights.04-0.78.hdf5',
    lr=0.0000001,
)

In [ ]:
encoder.model.summary()

In [ ]:
plot_losses = PlotLosses()
checkpoint = keras.callbacks.ModelCheckpoint(
    './convnet_weights/weights.{epoch:02d}-{val_loss:.2f}.hdf5', 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=False, 
    save_weights_only=True, 
    mode='auto', 
    period=1,
)
callbacks = [plot_losses, checkpoint]

In [ ]:
encoder.train(100, callbacks)

## Encoding

In [ ]:
encoder = LSTMEncoder(
    hidden_layers=layers,
    source_path='./features/lstm/2_steps/',
    weights_path='./convnet_weights/weights.02-0.78.hdf5',
)

In [ ]:
_ = encoder.encode('train', './features/lstm/encoded/features_train.npy')
_ = encoder.encode('dev', './features/lstm/encoded/features_dev.npy')

## Predictions

In [ ]:
encoder = LSTMEncoder(
    source_path='./features/lstm/2_steps/',
    regularizer=l2, 
    lr=0.000001,
    weights_path='./convnet_weights/lstm/base/weights.19-1.13.hdf5',
)

In [ ]:
encoder.predict('dev')